# NAIVE LEARNING

In [1]:
import os
import wandb

from sage.config import *
from sage.training.runner import run
from utils.misc import seed_everything, get_today
from IPython.display import clear_output

In [2]:
cfg = load_config()
seed_everything(cfg.seed)
cfg.registration = 'mni'
cfg.unused_src = ['IXI', 'Dallas', 'Oasis1']
cfg.plot = False

cfg.augment = True
cfg.epochs = 100
cfg.model_name = 'resnet_no_maxpool'
cfg.early_patience = 20
# cfg.start_channels = 8

In [3]:
name = 'sweep test'
cfg.RESULT_PATH = os.path.join(cfg.RESULT_PATH, name + get_today())

In [4]:
wandb.login()

wandb: Currently logged in as: 1pha (use `wandb login --relogin` to force relogin)


True

In [5]:
wandb.init(project='3d_smri',
           config=vars(cfg),
           name=name
  )

In [6]:
run(cfg)

Model Resnet_no_maxpool is selected.
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 16, 48, 48, 48]           5,488
       BatchNorm3d-2       [-1, 16, 48, 48, 48]              32
              ReLU-3       [-1, 16, 48, 48, 48]               0
            Conv3d-4       [-1, 16, 48, 48, 48]           6,912
       BatchNorm3d-5       [-1, 16, 48, 48, 48]              32
              ReLU-6       [-1, 16, 48, 48, 48]               0
            Conv3d-7       [-1, 16, 48, 48, 48]           6,912
       BatchNorm3d-8       [-1, 16, 48, 48, 48]              32
              ReLU-9       [-1, 16, 48, 48, 48]               0
       BasicBlock-10       [-1, 16, 48, 48, 48]               0
           Conv3d-11       [-1, 32, 24, 24, 24]          13,824
      BatchNorm3d-12       [-1, 32, 24, 24, 24]              64
             ReLU-13       [-1, 32, 24, 24, 24]               0
  

SystemError: <built-in function ResampleImageFilter_Execute> returned a result with an error set

In [ ]:
wandb.finish()

## SWEEP

In [2]:
import wandb

sweep_config = {
    "name" : "Resnet",
    "method" : "grid", #grid, random
    "metric": { #목표로 삶을 매트릭에 대한 정보를 입력합니다.
        "name": "valid_mae",
        "goal": "minimize"
    },
    "parameters" : { #실험해볼 하이퍼 파라미터의 조합입니다.
        "optimizer" : {
            "values" : ['adam', 'adamW']
        },
        "weight_decay" :{
            "values": [0, 0.01]
        },
        "start_channels" :{
            "values": [8, 16, 32]
        }
      }
    }

sweep_id = wandb.sweep(sweep_config, project='3d_smri')

Create sweep with ID: i7iv565s
Sweep URL: https://wandb.ai/1pha/3d_smri/sweeps/i7iv565s


In [3]:
def run_sweep():
    
    with wandb.init():

        cfg = load_config()
        seed_everything(cfg.seed)
        cfg.registration = 'mni'
        cfg.unused_src = []
        cfg.plot = False

        cfg.augment = True
        cfg.epochs = 100
        cfg.model_name = 'resnet_no_maxpool'
        cfg.early_patience = 20
        
        _cfg = wandb.config
        __cfg = dict()
        __cfg.update(_cfg)
        cfg.update(__cfg)
        
        name = f'SWEEP_OPT-{cfg.optimizer}_WD{cfg.weight_decay}_SC{cfg.start_channels}'
        cfg.RESULT_PATH = os.path.join(cfg.RESULT_PATH, name + get_today())
        
        wandb.run.name = name
        wandb.config = cfg
        
        run(cfg)
        
        clear_output()

In [ ]:
wandb.agent(sweep_id, function=run_sweep)

wandb: Agent Starting Run: 3gz020sk with config:
wandb: 	optimizer: adam
wandb: 	start_channels: 8
wandb: 	weight_decay: 0
wandb: Currently logged in as: 1pha (use `wandb login --relogin` to force relogin)


Model Resnet_no_maxpool is selected.
TOTAL TRAIN 2282 | VALID 286
Epoch 1 / 100, BEST MAE inf
[train] 232.2 sec [valid] 9.4 sec 
TRAIN :: LOSS 3425.387 | RMSE 58.52 | MAE 55.53 | R2 -8.84 | CORR 0.29
VALID :: LOSS 3259.163 | RMSE 57.11 | MAE 53.88 | R2 -7.79 | CORR 0.63
Saving ...
Epoch 2 / 100, BEST MAE 53.881
[train] 238.5 sec [valid] 9.1 sec 
TRAIN :: LOSS 3320.314 | RMSE 57.62 | MAE 54.64 | R2 -8.54 | CORR 0.48
VALID :: LOSS 3148.880 | RMSE 56.13 | MAE 52.96 | R2 -7.49 | CORR 0.69
Saving ...
Epoch 3 / 100, BEST MAE 52.962
[train] 231.3 sec [valid] 8.8 sec 
TRAIN :: LOSS 3233.732 | RMSE 56.86 | MAE 53.91 | R2 -8.29 | CORR 0.56
VALID :: LOSS 3076.361 | RMSE 55.48 | MAE 52.34 | R2 -7.29 | CORR 0.72
Saving ...
Epoch 4 / 100, BEST MAE 52.339
[train] 221.0 sec [valid] 9.0 sec 
TRAIN :: LOSS 3148.170 | RMSE 56.11 | MAE 53.17 | R2 -8.05 | CORR 0.61
VALID :: LOSS 3027.780 | RMSE 55.04 | MAE 51.96 | R2 -7.16 | CORR 0.74
Saving ...
Epoch 5 / 100, BEST MAE 51.959
[train] 232.5 sec [valid] 9.0 

# UNLEARNING

In [1]:
import os
import wandb

from sage.config import *
from sage.training import unlearner
from utils.misc import seed_everything, get_today

In [2]:
cfg = load_config()
seed_everything(cfg.seed)
cfg.registration = 'mni'
cfg.unused_src = ['Oasis1', 'Oasis3']

cfg.unlearn = True
cfg.unlearn_cfg.encoder.name = 'resnet'

cfg.unlearn_cfg.opt_conf.point = 50
# cfg.unlearn_cfg.opt_conf.use = False
cfg.unlearn_cfg.domainer.num_dbs = 4 - len(cfg.unused_src)

In [3]:
name = 'RES(128) 2DB CONF50 '
cfg.RESULT_PATH = os.path.join(cfg.RESULT_PATH, name + get_today())

In [4]:
wandb.login()
wandb.init(project='3d_smri_unlearning',
           config=vars(cfg),
           name=name
  )

wandb: Currently logged in as: 1pha (use `wandb login --relogin` to force relogin)


In [5]:
unlearner.run(cfg)

Use cuda:0 as a device.
Output from encoder is 128.
TOTAL TRAIN 448 | VALID 112
Epoch 1 / 100, BEST MAE inf
 opt_age: True  opt_dom: True  opt_conf: False 
[train] 16.2 sec [valid] 2.5 sec 
	TRAIN :: 
	REG  3154.26 | DOM     0.69 | CONF    0.00
	ACC      0.52 | AUC      0.51
	RMSE    56.16 | MAE     53.17 | R2      -8.61 | CORR     0.20
	VALID :: 
	REG  2692.40 | DOM     0.69 | CONF    0.00
	ACC      0.62 | AUC      0.50
	RMSE    51.89 | MAE     49.17 | R2      -8.81 | CORR    -0.22
Saving ...
Epoch 2 / 100, BEST MAE 49.171
 opt_age: True  opt_dom: True  opt_conf: False 
[train] 15.2 sec [valid] 2.6 sec 
	TRAIN :: 
	REG  3119.16 | DOM     0.69 | CONF    0.00
	ACC      0.67 | AUC      0.67
	RMSE    55.85 | MAE     52.85 | R2      -8.50 | CORR     0.35
	VALID :: 
	REG  2672.48 | DOM     0.69 | CONF    0.00
	ACC      0.62 | AUC      0.50
	RMSE    51.70 | MAE     48.97 | R2      -8.73 | CORR     0.32
Saving ...
Epoch 3 / 100, BEST MAE 48.971
 opt_age: True  opt_dom: True  opt_conf: False 


[train] 16.4 sec [valid] 2.6 sec 
	TRAIN :: 
	REG    34.30 | DOM     0.68 | CONF    0.00
	ACC      0.62 | AUC      0.59
	RMSE     5.86 | MAE      4.59 | R2       0.90 | CORR     0.96
	VALID :: 
	REG    82.68 | DOM     0.67 | CONF    0.00
	ACC      0.69 | AUC      0.59
	RMSE     9.09 | MAE      7.48 | R2       0.70 | CORR     0.87
Epoch 22 / 100, BEST MAE 7.165
 opt_age: True  opt_dom: True  opt_conf: False 
[train] 16.4 sec [valid] 2.3 sec 
	TRAIN :: 
	REG    29.49 | DOM     0.68 | CONF    0.00
	ACC      0.59 | AUC      0.56
	RMSE     5.43 | MAE      4.22 | R2       0.91 | CORR     0.96
	VALID :: 
	REG    82.44 | DOM     0.66 | CONF    0.00
	ACC      0.68 | AUC      0.58
	RMSE     9.08 | MAE      7.39 | R2       0.70 | CORR     0.87
Epoch 23 / 100, BEST MAE 7.165
 opt_age: True  opt_dom: True  opt_conf: False 
[train] 14.8 sec [valid] 2.4 sec 
	TRAIN :: 
	REG    27.47 | DOM     0.68 | CONF    0.00
	ACC      0.60 | AUC      0.57
	RMSE     5.24 | MAE      4.10 | R2       0.92 | CORR     

[train] 14.4 sec [valid] 2.4 sec 
	TRAIN :: 
	REG    21.47 | DOM     0.67 | CONF    0.00
	ACC      0.62 | AUC      0.60
	RMSE     4.63 | MAE      3.61 | R2       0.93 | CORR     0.97
	VALID :: 
	REG   106.59 | DOM     0.66 | CONF    0.00
	ACC      0.66 | AUC      0.56
	RMSE    10.32 | MAE      8.44 | R2       0.61 | CORR     0.87
Epoch 42 / 100, BEST MAE 6.593
 opt_age: True  opt_dom: True  opt_conf: False 
[train] 14.9 sec [valid] 2.3 sec 
	TRAIN :: 
	REG    22.16 | DOM     0.67 | CONF    0.00
	ACC      0.62 | AUC      0.60
	RMSE     4.71 | MAE      3.67 | R2       0.93 | CORR     0.97
	VALID :: 
	REG    89.43 | DOM     0.66 | CONF    0.00
	ACC      0.69 | AUC      0.59
	RMSE     9.46 | MAE      7.70 | R2       0.67 | CORR     0.87
Epoch 43 / 100, BEST MAE 6.593
 opt_age: True  opt_dom: True  opt_conf: False 
[train] 14.7 sec [valid] 2.4 sec 
	TRAIN :: 
	REG    22.15 | DOM     0.67 | CONF    0.00
	ACC      0.62 | AUC      0.61
	RMSE     4.71 | MAE      3.70 | R2       0.93 | CORR     

train_rmse,4.69168
train_mae,3.7556
train_r2,0.93295
train_corr,0.966
valid_rmse,8.84719
valid_mae,7.47089
valid_r2,0.71491
valid_corr,0.8853
train_acc,0.62723
train_auc,0.60565
valid_acc,0.65179


train_rmse,█████▇▇▇▆▅▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_mae,█████▇▇▇▆▅▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_r2,▁▁▁▁▂▂▃▃▄▅▆▆▇███████████████████████████
train_corr,▁▂▄▅▆▇▇▇████████████████████████████████
valid_rmse,██████▇▇▇▆▄▃▄▄▁▂▁▁▁▁▂▁▁▁▂▁▂▁▂▁▁▁▁▁▂▁▂▁▂▁
valid_mae,██████▇█▇▆▄▃▄▄▁▂▁▁▁▁▂▁▁▁▂▁▂▁▂▁▁▁▁▁▂▁▂▁▂▁
valid_r2,▁▁▁▂▁▂▂▂▂▄▆▇▆▆██████████████████████████
valid_corr,▁▄▇▇▇▇█▇▇████▇██████████████████████████
train_acc,▁▇█▅▂▄▅▄▅▄▄▆▅▅▄▅▅▄▄▄▅▄▆▅▆▆▅▅▆▅▆▅▅▅▅▅▆▅▅▆
train_auc,▁▇█▅▂▄▄▃▄▃▃▅▄▄▃▄▄▃▃▃▅▃▅▄▅▅▄▄▅▅▅▄▄▄▅▄▅▄▄▅
valid_acc,▅▅▆▁▅▅▆▅▅▅▆▆▅▅▇▆▇▇▇█▆▆█▇▆▆▆▇▆▇▆▇▆▇▅▇▆▇▇▆


{'encoder': ResNet(
   (conv1): Conv3d(1, 16, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
   (bn1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
       (bn1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
       (bn2): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
   )
   (layer2): Sequential(
     (0): BasicBlock(
       (conv1): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
       (bn1): BatchNorm3d(32, eps=1e

--- CUT

In [1]:
import os
import wandb

from sage.config import *
from sage.training import unlearner
from utils.misc import seed_everything, get_today

In [2]:
cfg = load_config()
seed_everything(cfg.seed)
cfg.registration = 'mni'
cfg.unused_src = ['Oasis1', 'Oasis3']

cfg.unlearn = True
cfg.unlearn_cfg.encoder.name = 'resnet'

cfg.unlearn_cfg.opt_conf.point = 50
cfg.early_patience = 40
cfg.epochs = 150
# cfg.unlearn_cfg.opt_conf.use = False
cfg.unlearn_cfg.domainer.num_dbs = 4 - len(cfg.unused_src)

In [4]:
name = 'RES(128) 2DB CONF50 '
cfg.RESULT_PATH = os.path.join(cfg.RESULT_PATH, name + get_today())

In [5]:
wandb.login()
wandb.init(project='3d_smri_unlearning',
           config=vars(cfg),
           name=name
  )

wandb: Currently logged in as: 1pha (use `wandb login --relogin` to force relogin)


In [6]:
unlearner.run(cfg)

Use cuda:0 as a device.
Output from encoder is 128.
TOTAL TRAIN 448 | VALID 112
Epoch 1 / 150, BEST MAE inf
 opt_age: True  opt_dom: True  opt_conf: False 
[train] 23.4 sec [valid] 3.8 sec 
	TRAIN :: 
	REG  3154.27 | DOM     0.69 | CONF    0.00
	ACC      0.52 | AUC      0.51
	RMSE    56.16 | MAE     53.17 | R2      -8.61 | CORR     0.20
	VALID :: 
	REG  2692.52 | DOM     0.69 | CONF    0.00
	ACC      0.62 | AUC      0.50
	RMSE    51.89 | MAE     49.17 | R2      -8.81 | CORR    -0.20
Saving ...
Epoch 2 / 150, BEST MAE 49.173
 opt_age: True  opt_dom: True  opt_conf: False 
[train] 19.3 sec [valid] 2.7 sec 
	TRAIN :: 
	REG  3119.16 | DOM     0.69 | CONF    0.00
	ACC      0.67 | AUC      0.67
	RMSE    55.85 | MAE     52.85 | R2      -8.50 | CORR     0.35
	VALID :: 
	REG  2672.12 | DOM     0.69 | CONF    0.00
	ACC      0.62 | AUC      0.50
	RMSE    51.69 | MAE     48.97 | R2      -8.73 | CORR     0.36
Saving ...
Epoch 3 / 150, BEST MAE 48.967
 opt_age: True  opt_dom: True  opt_conf: False 


[train] 16.1 sec [valid] 2.5 sec 
	TRAIN :: 
	REG    43.32 | DOM     0.68 | CONF    0.00
	ACC      0.63 | AUC      0.61
	RMSE     6.58 | MAE      5.15 | R2       0.87 | CORR     0.95
	VALID :: 
	REG   151.44 | DOM     0.67 | CONF    0.00
	ACC      0.64 | AUC      0.53
	RMSE    12.31 | MAE     10.28 | R2       0.45 | CORR     0.87
Epoch 22 / 150, BEST MAE 6.713
 opt_age: True  opt_dom: True  opt_conf: False 
[train] 15.5 sec [valid] 2.6 sec 
	TRAIN :: 
	REG    30.87 | DOM     0.68 | CONF    0.00
	ACC      0.59 | AUC      0.56
	RMSE     5.56 | MAE      4.31 | R2       0.91 | CORR     0.96
	VALID :: 
	REG   301.19 | DOM     0.66 | CONF    0.00
	ACC      0.62 | AUC      0.51
	RMSE    17.35 | MAE     14.83 | R2      -0.10 | CORR     0.83
Epoch 23 / 150, BEST MAE 6.713
 opt_age: True  opt_dom: True  opt_conf: False 
[train] 15.5 sec [valid] 2.5 sec 
	TRAIN :: 
	REG    27.29 | DOM     0.68 | CONF    0.00
	ACC      0.60 | AUC      0.57
	RMSE     5.22 | MAE      4.14 | R2       0.92 | CORR     

[train] 15.2 sec [valid] 2.4 sec 
	TRAIN :: 
	REG    20.70 | DOM     0.67 | CONF    0.00
	ACC      0.61 | AUC      0.59
	RMSE     4.55 | MAE      3.52 | R2       0.94 | CORR     0.97
	VALID :: 
	REG    96.44 | DOM     0.66 | CONF    0.00
	ACC      0.67 | AUC      0.57
	RMSE     9.82 | MAE      8.02 | R2       0.65 | CORR     0.88
Epoch 42 / 150, BEST MAE 6.381
 opt_age: True  opt_dom: True  opt_conf: False 
[train] 15.0 sec [valid] 2.5 sec 
	TRAIN :: 
	REG    20.25 | DOM     0.67 | CONF    0.00
	ACC      0.62 | AUC      0.60
	RMSE     4.50 | MAE      3.49 | R2       0.94 | CORR     0.97
	VALID :: 
	REG   163.57 | DOM     0.66 | CONF    0.00
	ACC      0.67 | AUC      0.57
	RMSE    12.79 | MAE     10.82 | R2       0.40 | CORR     0.88
Epoch 43 / 150, BEST MAE 6.381
 opt_age: True  opt_dom: True  opt_conf: False 
[train] 16.1 sec [valid] 2.6 sec 
	TRAIN :: 
	REG    20.37 | DOM     0.67 | CONF    0.00
	ACC      0.62 | AUC      0.60
	RMSE     4.51 | MAE      3.49 | R2       0.94 | CORR     

[train] 20.3 sec [valid] 2.3 sec 
	TRAIN :: 
	REG    29.47 | DOM     0.69 | CONF  112.29
	ACC      0.56 | AUC      0.53
	RMSE     5.43 | MAE      4.26 | R2       0.91 | CORR     0.96
	VALID :: 
	REG    63.06 | DOM     0.68 | CONF  112.03
	ACC      0.62 | AUC      0.50
	RMSE     7.94 | MAE      6.27 | R2       0.77 | CORR     0.88
Epoch 62 / 150, BEST MAE 6.022
 opt_age: True  opt_dom: True  opt_conf: True 
[train] 20.2 sec [valid] 2.4 sec 
	TRAIN :: 
	REG    30.83 | DOM     0.69 | CONF  112.43
	ACC      0.55 | AUC      0.52
	RMSE     5.55 | MAE      4.43 | R2       0.91 | CORR     0.95
	VALID :: 
	REG    93.86 | DOM     0.69 | CONF  111.45
	ACC      0.61 | AUC      0.49
	RMSE     9.69 | MAE      8.02 | R2       0.66 | CORR     0.88
Epoch 63 / 150, BEST MAE 6.022
 opt_age: True  opt_dom: True  opt_conf: True 
[train] 20.1 sec [valid] 2.4 sec 
	TRAIN :: 
	REG    29.09 | DOM     0.69 | CONF  111.94
	ACC      0.52 | AUC      0.49
	RMSE     5.39 | MAE      4.23 | R2       0.91 | CORR     0.

[train] 22.1 sec [valid] 2.5 sec 
	TRAIN :: 
	REG    28.17 | DOM     0.69 | CONF  111.86
	ACC      0.54 | AUC      0.52
	RMSE     5.31 | MAE      4.18 | R2       0.91 | CORR     0.96
	VALID :: 
	REG    62.99 | DOM     0.68 | CONF  111.63
	ACC      0.63 | AUC      0.53
	RMSE     7.94 | MAE      6.25 | R2       0.77 | CORR     0.88
Epoch 82 / 150, BEST MAE 6.022
 opt_age: True  opt_dom: True  opt_conf: True 
[train] 22.6 sec [valid] 2.7 sec 
	TRAIN :: 
	REG    25.86 | DOM     0.69 | CONF  111.70
	ACC      0.52 | AUC      0.51
	RMSE     5.09 | MAE      4.03 | R2       0.92 | CORR     0.96
	VALID :: 
	REG    58.36 | DOM     0.68 | CONF  111.78
	ACC      0.65 | AUC      0.56
	RMSE     7.64 | MAE      6.14 | R2       0.79 | CORR     0.89
Epoch 83 / 150, BEST MAE 6.022
 opt_age: True  opt_dom: True  opt_conf: True 
[train] 24.8 sec [valid] 3.0 sec 
	TRAIN :: 
	REG    24.99 | DOM     0.69 | CONF  111.68
	ACC      0.51 | AUC      0.49
	RMSE     5.00 | MAE      3.99 | R2       0.92 | CORR     0.

train_rmse,4.31897
train_mae,3.49816
train_r2,0.94318
train_corr,0.97175
valid_rmse,8.01246
valid_mae,6.11467
valid_r2,0.76617
valid_corr,0.87931
train_acc,0.53795
train_auc,0.50351
valid_acc,0.625


train_rmse,███▇▆▅▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_mae,███▇▆▅▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_r2,▁▁▁▃▄▆▇█████████████████████████████████
train_corr,▁▄▆▇████████████████████████████████████
valid_rmse,██▇▇▇▆▅▁▂▃▁▁▁▁▂▂▁▂▁▂▁▂▂▂▂▁▁▁▂▂▂▁▁▁▁▁▁▁▁▁
valid_mae,██▇▇▇▆▅▁▂▂▁▁▁▁▂▂▁▂▁▂▁▂▂▂▂▁▁▁▂▂▂▁▁▁▁▁▁▁▁▁
valid_r2,▁▁▂▂▂▅▅██▇██████████████████████████████
valid_corr,▁▇▇█▇███████████████████████████████████
train_acc,▁█▃▄▄▄▄▅▄▄▅▄▅▆▅▅▅▅▆▅▅▂▂▃▂▃▂▂▁▃▃▂▁▂▁▃▃▃▂▂
train_auc,▂█▄▄▃▃▄▄▄▄▄▄▄▆▅▅▅▅▅▅▅▂▂▃▂▂▂▂▁▂▃▂▁▂▁▃▂▂▂▁
valid_acc,▆▆▁▆▆▆▆▇█▆██▇▇▆▆▇▇▇▆▇█▆▆▆▆▅▆▆▆▆▆▆▆▆▆▆▆▆▆


{'encoder': ResNet(
   (conv1): Conv3d(1, 16, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
   (bn1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
       (bn1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
       (bn2): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
   )
   (layer2): Sequential(
     (0): BasicBlock(
       (conv1): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
       (bn1): BatchNorm3d(32, eps=1e